In [160]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("ticks")
import pandas as pd

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import SpectralClustering
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.metrics import calinski_harabasz_score

In [27]:
df = pd.read_csv('processed_data/all_data.csv')
df.head()

,CPF,Potencial Bruto,Raciocínio,Social,Motivacional,Cultura pontuação,atributo-Capacidade analítica,atributo-Pensamento conceitual,atributo-Reflexão,atributo-Pensamento criativo,...,perfil-Facilitação,perfil-Flexibilidade,perfil-Estabilidade emocional,perfil-Ambição,perfil-Iniciativa,perfil-Assertividade,perfil-Tomada de riscos,Área,AveragePerformance,ModePerformance
0,678.042.935-18,53.65,65.70,37.65,56.75,32.33,59.67,49.44,63.08,56.26,...,41.95,55.59,78.32,60.14,41.95,8.74,55.59,Operações,2.50,2.0
1,089.324.175-12,NaN,60.78,NaN,NaN,35.13,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Logística,2.00,2.0
2,587.416.093-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Financeiro,2.00,2.0
3,375.410.982-13,22.46,15.71,5.67,56.75,23.36,11.43,18.25,13.14,16.54,...,32.86,46.50,46.50,32.86,51.05,32.86,41.95,Operações,2.75,3.0
4,652.134.809-42,NaN,56.15,NaN,NaN,62.03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pessoas,1.75,1.0


In [45]:
df.columns

Index(['CPF', 'Potencial Bruto', 'Raciocínio', 'Social', 'Motivacional',
       'Cultura pontuação', 'atributo-Capacidade analítica',
       'atributo-Pensamento conceitual', 'atributo-Reflexão',
       'atributo-Pensamento criativo', 'atributo-Planejamento e organização',
       'atributo-Comunicação', 'atributo-Consideração pelos outros',
       'atributo-Influência', 'atributo-Sociabilidade', 'atributo-Facilitação',
       'atributo-Flexibilidade', 'atributo-Estabilidade emocional',
       'atributo-Ambição', 'atributo-Iniciativa', 'atributo-Assertividade',
       'atributo-Tomada de riscos', 'perfil-Capacidade analítica',
       'perfil-Pensamento conceitual', 'perfil-Reflexão',
       'perfil-Pensamento criativo', 'perfil-Planejamento e organização',
       'perfil-Comunicação', 'perfil-Consideração pelos outros',
       'perfil-Influência', 'perfil-Sociabilidade', 'perfil-Facilitação',
       'perfil-Flexibilidade', 'perfil-Estabilidade emocional',
       'perfil-Ambição', 'perfi

Small amount of variables with KNN imputation

In [124]:
X = df[['Raciocínio','Social','Motivacional','Cultura pontuação']]
X = KNNImputer(missing_values=np.nan,n_neighbors = 5,add_indicator=False).fit_transform(X)
X = StandardScaler().fit_transform(X)

In [125]:
cluster_result = {}
for n_cluster in [2,5,8,10]:
    spec_c = SpectralClustering(n_clusters=n_cluster, assign_labels='discretize',random_state=0).fit(X)
    score = silhouette_score(X, spec_c.labels_, metric='euclidean')
    print('Finished n_cluster = ',n_cluster)
    cluster_result[n_cluster] = score
print(cluster_result)

Finished n_cluster =  2
Finished n_cluster =  5
Finished n_cluster =  8
Finished n_cluster =  10
{2: 0.3037144639652581, 5: 0.2618193912313312, 8: 0.2501253223387485, 10: 0.2390405987089542}


In [169]:
for min_samples in [2,3,4]:
    for eps in [1,1.005,1.01,1.02,1.03,1.04,1.05,1.09,1.1]:
        spec_c = DBSCAN(min_samples = min_samples,eps=eps).fit(X)
        silhou = silhouette_score(X, spec_c.labels_, metric='euclidean')
        calins = calinski_harabasz_score(X, spec_c.labels_)
        if len([x for x in set(spec_c.labels_) if x >=0]) > 2 and silhou > 0.2:
            print('Params {0}-{1} has {2} clusters and {3}-{4} score'.format(min_samples,eps,len([x for x in set(spec_c.labels_) if x >=0]),round(silhou,2),round(calins,0)))

Params 2-1.09 has 7 clusters and 0.22-10.0 score
Params 2-1.1 has 7 clusters and 0.22-10.0 score
Params 3-1 has 5 clusters and 0.24-13.0 score
Params 3-1.005 has 5 clusters and 0.24-13.0 score
Params 3-1.01 has 5 clusters and 0.24-14.0 score
Params 3-1.03 has 4 clusters and 0.25-14.0 score
Params 3-1.04 has 4 clusters and 0.25-14.0 score
Params 3-1.05 has 3 clusters and 0.29-11.0 score
Params 3-1.09 has 3 clusters and 0.29-11.0 score
Params 3-1.1 has 3 clusters and 0.29-11.0 score
Params 4-1.02 has 3 clusters and 0.28-12.0 score
Params 4-1.03 has 3 clusters and 0.28-11.0 score
Params 4-1.04 has 3 clusters and 0.3-13.0 score
Params 4-1.05 has 3 clusters and 0.31-13.0 score
Params 4-1.09 has 3 clusters and 0.32-11.0 score


Big number of variables, just rows without Nan

In [106]:
X = df[['Potencial Bruto', 'Raciocínio', 'Social', 'Motivacional',
       'Cultura pontuação', 'atributo-Capacidade analítica',
       'atributo-Pensamento conceitual', 'atributo-Reflexão',
       'atributo-Pensamento criativo', 'atributo-Planejamento e organização',
       'atributo-Comunicação', 'atributo-Consideração pelos outros',
       'atributo-Influência', 'atributo-Sociabilidade', 'atributo-Facilitação',
       'atributo-Flexibilidade', 'atributo-Estabilidade emocional',
       'atributo-Ambição', 'atributo-Iniciativa', 'atributo-Assertividade',
       'atributo-Tomada de riscos', 'perfil-Capacidade analítica',
       'perfil-Pensamento conceitual', 'perfil-Reflexão',
       'perfil-Pensamento criativo', 'perfil-Planejamento e organização',
       'perfil-Comunicação', 'perfil-Consideração pelos outros',
       'perfil-Influência', 'perfil-Sociabilidade', 'perfil-Facilitação',
       'perfil-Flexibilidade', 'perfil-Estabilidade emocional',
       'perfil-Ambição', 'perfil-Iniciativa', 'perfil-Assertividade',
       'perfil-Tomada de riscos']]
X = X.dropna(axis=0,how='any')
X = StandardScaler().fit_transform(X)

In [107]:
cluster_result = {}
for n_cluster in [2,3,4,5,8,10]:
    spec_c = SpectralClustering(n_clusters=n_cluster, assign_labels='discretize',random_state=0).fit(X)
    score = silhouette_score(X, spec_c.labels_, metric='euclidean')
    print('Finished n_cluster = ',n_cluster)
    cluster_result[n_cluster] = score
print(cluster_result)

Finished n_cluster =  2
Finished n_cluster =  3
Finished n_cluster =  4
Finished n_cluster =  5
Finished n_cluster =  8
Finished n_cluster =  10
{2: 0.12781259869240255, 3: 0.01689871607133735, 4: -0.06739386418642711, 5: -0.0712034535228569, 8: -0.09494512193323587, 10: -0.10020282256693462}
